Creating a first, very simple version of the model that takes in Spotify features of a track and predicts the maximum position it will achieve in the charts within one month. 

# Imports

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import  RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import  RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import pickle

# paths
datapath = "../../data/"
kaggle_set_full = "external/Database to calculate popularity.csv"
kaggle_set_final = "external/Final database.csv"
nmf_test_set = "interim/test_new_music_friday.parquet"

Note: Karlijn has found some indications that there are mistakes in the final database (e.g. duplicates where it is unclear which max popularity is right, which is important for us to know because it is the target variable that we fit our models on). For future models, this data preprocessing should be done correctly from the 'Database to calculate popularity.csv', but for now I will go with the already prepared dataset to get a working model quickly to try to implement on google cloud already. 

In [2]:
df = pd.read_csv(datapath+kaggle_set_final)
df.head()

/Users/emieldeheij/opt/anaconda3/envs/spotifAI/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (7,8,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Country,Uri,Popularity,Title,Artist,Album/Single,Genre,Artist_followers,Explicit,Album,...,Sweden,Switzerland,Taiwan,Turkey,UK,USA,Popu_max,Top10_dummy,Top50_dummy,Cluster
0,Global,https://open.spotify.com/track/6FyRXC8tJUh863J...,31833.95,adan y eva,Paulo Londra,single,argentine hip hop,1.14271e+07,False,Adan y Eva,...,0,0,0,0,0,0,10,1.0,1.0,global
1,USA,https://open.spotify.com/track/6FyRXC8tJUh863J...,8.00,adan y eva,Paulo Londra,single,argentine hip hop,1.14271e+07,False,Adan y Eva,...,0,0,0,0,0,1,191,0.0,0.0,english speaking and nordic
2,Argentina,https://open.spotify.com/track/6FyRXC8tJUh863J...,76924.40,adan y eva,Paulo Londra,single,argentine hip hop,1.14271e+07,False,Adan y Eva,...,0,0,0,0,0,0,1,1.0,1.0,spanish speaking
3,Belgium,https://open.spotify.com/track/6FyRXC8tJUh863J...,849.60,adan y eva,Paulo Londra,single,argentine hip hop,1.14271e+07,False,Adan y Eva,...,0,0,0,0,0,0,126,0.0,0.0,english speaking and nordic
4,Switzerland,https://open.spotify.com/track/6FyRXC8tJUh863J...,20739.10,adan y eva,Paulo Londra,single,argentine hip hop,1.14271e+07,False,Adan y Eva,...,0,1,0,0,0,0,21,0.0,1.0,english speaking and nordic


In [3]:
new_data = pd.read_parquet(datapath+nmf_test_set)
new_data.head()

,date_of_scrape,artist,name,track_id,track_popularity,date_added,spotify_url,danceability,energy,key,...,valence,tempo,duration_ms,time_signature,release_date,explicit,artist_ids,followers,artist_popularity,max_position
0,1631059200000,Drake ft. Lil Baby,Girls Want Girls (with Lil Baby),37Nqx7iavZpotJSDXZWbJ3,88,2021-09-03,https://open.spotify.com/track/37Nqx7iavZpotJS...,0.767,0.438,7,...,0.381,86.975,221980,4,2021-09-03,True,"[3TVXtAsR1Inumwj472S9r4, 5f7VJjfbwm532GiveGC0ZK]",65007711,96,45
1,1631059200000,Kanye West,Hurricane,0RbW8kWozrVSIGb4V13o0o,90,2021-09-03,https://open.spotify.com/track/0RbW8kWozrVSIGb...,0.587,0.561,0,...,0.351,80.009,243187,4,2021-08-29,False,[5K4W6rqBFWDnAN6FQUkS6x],14997901,98,66
2,1631059200000,Meek Mill ft. Lil Uzi Vert,Blue Notes 2 (feat. Lil Uzi Vert),488HG8lR11JswT2YgcWGRB,73,2021-09-03,https://open.spotify.com/track/488HG8lR11JswT2...,0.748,0.591,2,...,0.192,74.958,230400,4,2021-09-01,True,"[20sxb77xiYeusSH8cVdatc, 4O15NlyKLIASxsJ0PrXPfz]",17274352,85,173
3,1631059200000,Tokischa ft. ROSALÍA,Linda,1ahCrpeTt94LL7y1aXw0Y8,71,2021-09-03,https://open.spotify.com/track/1ahCrpeTt94LL7y...,0.891,0.751,2,...,0.711,115.046,144019,4,2021-09-01,True,"[2p4aN0Uxkk3iT3HK0cJ2cJ, 7ltDVBr6mKbRvohxheJ9h1]",4857723,72,82
4,1631059200000,Charli XCX,Good Ones,6PZpNMstpIiRenGK5UyG5D,74,2021-09-03,https://open.spotify.com/track/6PZpNMstpIiRenG...,0.764,0.672,7,...,0.789,129.049,136614,4,2021-09-02,False,[25uiPmTg16RbhZWAqwLBy5],2406962,79,183


# Minimal data cleaning

In [4]:
## TODO: add column df["days_between_release_and_max_position"] 
# (look in Karlijn's notebook for;) df_highest_first['days_to_highest_rank'] = (df_highest_first['date_highest_rank'] - df_highest_first['Release_date']).dt.days


In [5]:
# let's use this set of predictors for the max position
predictor_variables = [
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "duration_ms",
    "explicit",
    "followers",
]

## kaggle_set

In [6]:
df = df.drop(
    columns=["country"]
)  # drop column country (referring to the genre) to prevent duplicate column names

df = df.rename(columns=str.lower)  # lower case all column names

# only keep global information
df = df.loc[df["country"] == "Global"]

# only keep tracks that have achieved their max position within one month
# NOTE: we should create this variable ourselves from the full dataset
# df["days_between_release_and_max_position"] < 31

# NOTE 2: we should also filter out the tracks in this dataset
# that were released within one month before the moment the dataset is 'recorded'
# i.e. tracks that did not have the chance to reach their maximum position within one month
df = df.loc[df.days_since_release > 31]

# rename variables to match the scraped data from nmf
df = df.rename(
    columns={
        "artist_followers": "followers",
        "popu_max": "max_position",
        "acoustics": "acousticness",
        "liveliness": "liveness",
    }
)


len(df)

5337

In [7]:
df.isna().sum()

country         0
uri             0
popularity      0
title           0
artist          0
               ..
usa             0
max_position    0
top10_dummy     0
top50_dummy     0
cluster         0
Length: 150, dtype: int64

In [8]:
df = df.dropna(subset=['followers']) # drop the row with the single missing value

In [9]:
# column selection
X = df[predictor_variables]
y = df["max_position"]

## New data

In [10]:
# drop the randomly assigned max_position column
new_data = new_data.drop(
    columns=["max_position"]
)

# column selection
new_data = new_data[["track_id"]+predictor_variables]


In [11]:
new_data

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,explicit,followers
0,37Nqx7iavZpotJSDXZWbJ3,0.767,0.438,7,-8.726,0,0.2900,0.1810,0.000000,0.1420,0.381,86.975,221980,True,65007711
1,0RbW8kWozrVSIGb4V13o0o,0.587,0.561,0,-6.569,1,0.1460,0.0466,0.000000,0.1090,0.351,80.009,243187,False,14997901
2,488HG8lR11JswT2YgcWGRB,0.748,0.591,2,-6.234,0,0.3310,0.1520,0.000000,0.6400,0.192,74.958,230400,True,17274352
3,1ahCrpeTt94LL7y1aXw0Y8,0.891,0.751,2,-3.150,1,0.1240,0.0685,0.000598,0.1900,0.711,115.046,144019,True,4857723
4,6PZpNMstpIiRenGK5UyG5D,0.764,0.672,7,-5.947,0,0.0293,0.0720,0.000003,0.0806,0.789,129.049,136614,False,2406962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4aQUdTn1sp1aCAb2FcL6RK,0.745,0.592,6,-7.823,1,0.0625,0.0939,0.027100,0.2070,0.826,160.067,187000,False,701
96,11dudYRtXcIapaSWEC2d9b,0.702,0.630,4,-7.661,1,0.0802,0.1720,0.000300,0.7390,0.942,98.055,196684,True,23608
97,4tqa4KJcc7j20i9L0n4pr9,0.764,0.611,10,-8.284,1,0.0732,0.1850,0.000000,0.0844,0.671,130.039,157409,False,176
98,4uXJobWSESExklnrQqKwU5,0.580,0.127,9,-16.498,1,0.0493,0.8630,0.000007,0.0850,0.266,80.015,211453,True,14506


# Modelling
This part should be worked on! For now contains minimal effort for selection of models.

In [12]:
# randomly split the data
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.25,random_state=0)

# shape of train and test splits
train_x.shape, test_x.shape, train_y.shape, test_y.shape

# TODO: write testset to datafolder and keep that stable

((4002, 14), (1334, 14), (4002,), (1334,))

In [13]:
model_LR = LinearRegression()
model_RF = RandomForestRegressor()

In [14]:
scaler = StandardScaler()

pipeline_LR = Pipeline([('transformer', scaler), ('estimator', model_LR)])
pipeline_RF = Pipeline([('transformer', scaler), ('estimator', model_RF)])

# note: All scorer objects follow the convention that higher return values 
# are better than lower return values. Thus metrics which measure the distance 
# between the model and the data, like metrics.mean_squared_error, are available
# as neg_mean_squared_error which return the negated value of the metric.
LR_scores = cross_val_score(pipeline_LR, train_x, train_y, cv = 10, scoring = 'neg_root_mean_squared_error')
RF_scores = cross_val_score(pipeline_RF, train_x, train_y, cv = 10, scoring = 'neg_root_mean_squared_error')

print('Average RMSE over folds for Linear Regression:', round(abs(LR_scores.mean()), 3))
print('Average RMSE over folds for Random Forest Regressor:', round(abs(RF_scores.mean()), 3))

Average RMSE over folds for Linear Regression: 55.379
Average RMSE over folds for Random Forest Regressor: 53.756


At first sight, Random Forest Regressor seems to work best.

In [15]:
# fit the best model with the training data
model_RF.fit(train_x, train_y)

# predict the target on train and test data 
predict_train = model_RF.predict(train_x)
predict_test  = model_RF.predict(test_x)

# Root Mean Squared Error on train and test data
print('RMSE on train data: ', round(sqrt(mean_squared_error(train_y, predict_train)), 3))
print('RMSE on test data: ',  round(sqrt(mean_squared_error(test_y, predict_test)), 3))

# Mean Absolute Error on train and test date
print('MAE on train data: ', round(mean_absolute_error(train_y, predict_train), 3))
print('MAE on test data: ',  round(mean_absolute_error(test_y, predict_test), 3))

RMSE on train data:  20.797
RMSE on test data:  53.42
MAE on train data:  17.002
MAE on test data:  44.51


There is some heavy overfitting going on! A problem for later :)

In [16]:
# pickle model
modelpath = "../../models/"
modelname = "RFregressor_v1.p"
full_modelpath = modelpath+modelname
pickle.dump(model_RF, open(full_modelpath, 'wb'))


# simulate predictions on new music friday data

In [17]:
# load in pickle
loaded_model = pickle.load(open(full_modelpath, 'rb'))


In [18]:
new_data = new_data.assign(
    max_position=[round(pred) for pred in loaded_model.predict(new_data[predictor_variables])])


In [19]:
new_data.sort_values(by='max_position')

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,explicit,followers,max_position
0,37Nqx7iavZpotJSDXZWbJ3,0.767,0.438,7,-8.726,0,0.2900,0.1810,0.000000,0.1420,0.381,86.975,221980,True,65007711,17
6,6tvDAC8atHIjyLgvOEK1fK,0.803,0.649,6,-5.842,1,0.2190,0.1270,0.000065,0.0894,0.797,90.006,187896,False,35648474,29
1,0RbW8kWozrVSIGb4V13o0o,0.587,0.561,0,-6.569,1,0.1460,0.0466,0.000000,0.1090,0.351,80.009,243187,False,14997901,53
2,488HG8lR11JswT2YgcWGRB,0.748,0.591,2,-6.234,0,0.3310,0.1520,0.000000,0.6400,0.192,74.958,230400,True,17274352,57
5,4s7QLoImIwmPi9L6dq1nVW,0.568,0.560,9,-9.059,1,0.4200,0.2280,0.000003,0.3900,0.246,130.019,308756,True,56954188,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,3zpGIFghtnVlwMXWyKW474,0.668,0.856,1,-6.386,0,0.0453,0.0837,0.044600,0.0960,0.737,79.008,201787,False,1939938,128
51,2B58L5Us32hKTpc8BCA03A,0.576,0.732,0,-4.964,1,0.0393,0.0502,0.000004,0.1610,0.375,142.845,231551,False,74305,128
63,7xaJp99s5JRCwXFzsG1ESU,0.579,0.634,2,-5.497,1,0.0380,0.4440,0.000000,0.2250,0.234,134.811,216772,False,36113,129
76,5FKl1YJ1uIbJxmugKjJxxP,0.583,0.606,1,-5.897,0,0.0726,0.0838,0.000002,0.1420,0.603,173.949,159640,False,42827,131
